<a href="https://colab.research.google.com/github/preetmodi/Credit-Risk-Analytics/blob/main/Scorecard_logistic_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=po0RjVlNj4Egyvf%2FJc04yV3TNXHn2jD0TAVfHNKFWTo'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fww

In [2]:
!pip install git+https://github.com/CBravoR/scorecardpy

%cd /content/gdrive/My Drive/Data/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/CBravoR/scorecardpy to /tmp/pip-req-build-szz39cai
  Running command git clone --filter=blob:none --quiet https://github.com/CBravoR/scorecardpy /tmp/pip-req-build-szz39cai
  Resolved https://github.com/CBravoR/scorecardpy to commit 03ec28470391e5c950580ad9ac4877ca33f05fce
  Preparing metadata (setup.py) ... done
  Created wheel for scorecardpy: filename=scorecardpy-0.1.9.3-py3-none-any.whl size=59549 sha256=242923026a77c2bc9eacf6451c189f15c2284a6c4669dbed774d98da40e10079
  Stored in directory: /tmp/pip-ephem-wheel-cache-bpxfrjk8/wheels/61/36/7f/76fa329f36d896910f2251f5cfd54094c722e088b633e47f6f
Successfully built scorecardpy
/content/gdrive/My Drive/Data


In [3]:
# Package loading
import pandas as pd
import numpy as np
import scorecardpy as sc
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
%matplotlib inline
seed = 251256517

In [4]:
# Import the files as Pandas datasets
train_WoE = pd.read_csv('train_woe.csv')
test_WoE = pd.read_csv('test_woe.csv')

train_noWoE = pd.read_csv('train.csv')
test_noWoE = pd.read_csv('test.csv')

# Give breaks for WoE
with open('breaks_adj.json', 'r') as f:
    breaks_adj = json.load(f)
# breaks_adj = {'Address': [1.0,2.0,8.0,17.0],
#               'Age': [30.0,45.0,50.0],
#               'Creddebt': [1.0, 6.0],
#               'Employ': [4.0,14.0,22.0],
#               'Income': [30.0,40.0,80.0,140.0],
#               'Leverage': [8.0,16.0,22.0],
#               'MonthlyLoad': [0.1,0.2,0.30000000000000004,0.7000000000000001],
#               'OthDebt': [1.0,2.0,3.0],
#               'OthDebtRatio': [0.01,0.05,0.07,0.09,0.14]
#               }

# Apply breaks.
bins_adj = sc.woebin(train_noWoE, y="Default",
                     breaks_list=breaks_adj)
  

FileNotFoundError: ignored

In [5]:
from sklearn.linear_model import LogisticRegressionCV

logreg = LogisticRegressionCV(penalty='elasticnet', # Type of penalization l1 = lasso, l2 = ridge, elasticnet
                                     Cs = 10,        # How many parameters to try. Can also be a vector with parameters to try.
                                     tol=0.000001, # Tolerance for parameters
                                     cv = 3,     # How many CV folds to try. 3 or 5 should be enough.
                                     fit_intercept=True, # Use constant?
                                     class_weight='balanced', # Weights, see below
                                     random_state=20190301, # Random seed
                                     max_iter=100, # Maximum iterations
                                     verbose=2, # Show process. 1 is yes.
                                     solver = 'saga', # How to optimize.
                                     n_jobs = 2,      # Processes to use. Set to number of physical cores. 
                                     refit = True,     # If to retrain with the best parameter and all data after finishing.
                                     l1_ratios = np.arange(0, 1.01, 0.1), # The LASSO / Ridge ratios.
                                    )

# Training

In [ ]:
logreg.fit(X = train_WoE.iloc[:, 1:], # All rows and from the second var to end
                    y = train_WoE['Default'] # The target
                   )

#Parameter Inference

In [ ]:
coef_df = pd.concat([pd.DataFrame({'column': train_WoE.columns[np.r_[1:6,7]]}), 
                    pd.DataFrame(np.transpose(logreg.coef_))],
                    axis = 1
                   )

coef_df

In [ ]:
logreg.intercept_

In [ ]:
print(logreg.l1_ratio_)
print(logreg.C_)

#Applying on Test Set

In [ ]:
pred_class_test = logreg.predict(test_WoE.iloc[:, 1:].drop(columns="Income_woe"))
probs_test = logreg.predict_proba(test_WoE.iloc[:, 1:].drop(columns="Income_woe"))
print(probs_test[0:5], pred_class_test[0:5])

#Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix


# Calculate confusion matrix
confusion_matrix_cs = confusion_matrix(y_true = test_WoE['Default'],
                                        y_pred = pred_class_test)


# Turn matrix to percentages
confusion_matrix_cs = confusion_matrix_cs.astype('float') / confusion_matrix_cs.sum(axis=1)[:, np.newaxis]

# Turn to dataframe
df_cm = pd.DataFrame(
        confusion_matrix_cs, index=['Non Defaulter', 'Defaulter'], 
        columns=['Non Defaulter', 'Defaulter'], 
)

# Parameters of the image
figsize = (10,7)
fontsize=14

# Create image
fig = plt.figure(figsize=figsize)
heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')

# Make it nicer
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, 
                             ha='right', fontsize=fontsize)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45,
                             ha='right', fontsize=fontsize)

# Add labels
plt.ylabel('True label')
plt.xlabel('Predicted label')

# Plot!
plt.show()

# Create Scorecard

In [ ]:
loans_sc = sc.scorecard(bins_adj,         # bins from the WoE
                           logreg,  # Trained logistic regression
                           train_WoE.columns[np.r_[1:6,7]], # The column names in the trained LR
                           points0=750, # Base points
                           odds0=0.01, # Base odds bads:goods
                           pdo=50
                           ) # PDO 

In [ ]:
loans_sc

#Create Credit Score

In [ ]:
train_score = sc.scorecard_ply(train_noWoE, loans_sc, 
                               print_step=0)
test_score = sc.scorecard_ply(test_noWoE, loans_sc, 
                               print_step=0)